In [1]:
import pandas as pd
import os
import json
import re
import pandas as pd
from rapidfuzz import fuzz
import plotly.graph_objects as go




In [2]:
# load json reference/itemsNumber.json
with open('references/itemNum.json') as f:
    ITEMS_NUMBERS = json.load(f)
with open('references/itemStandard.json') as f:
    ITEMS_STANDARD = json.load(f)
with open('references/items.json') as f:
    ITEMS = json.load(f)


In [3]:
# Helper functions
# -------------------------------------------------------------------

def find_misplaced_items(items_list, items_numbers=ITEMS_NUMBERS):
    """
    Given:
      items_list: a list of dicts like [{"label": ..., "page": ..., "text": ...}, ...]
      items_numbers: a dict mapping each label to its numeric order (float).
    
    Returns:
      misplaced_items: a list of items that cause the sequence to be invalid
                       and must be removed so that the rest follow ascending item order
                       and non-decreasing page number.
      valid_sequence:  a list of items that remain in the correct ascending chain.
    
    We do this by:
      1) Convert each item to (item_order, page, original_index).
      2) Sort by (item_order, then page).
      3) Find the LIS in terms of page among those sorted items.
      4) Everything not in that LIS is 'misplaced'.
    """
    # ----------------------------------------------------------------
    #    Convert items to a structure: (order_val, page, original_index)
    #    where order_val is from ITEMS_NUMBERS[label].
    # ----------------------------------------------------------------
    expanded = []
    for idx, item in enumerate(items_list):
        label = item["label"]
        page = item["page"]
        # If label not in items_numbers, set an arbitrary large or handle gracefully
        # but presumably all should be in items_numbers
        order_val = items_numbers.get(label, 100)  
        expanded.append((order_val, page, idx))
    
    # ----------------------------------------------------------------
    #    Sort the expanded items primarily by order_val, secondarily by page.
    #    This ensures we handle them in ascending item order, then by page.
    # ----------------------------------------------------------------
    expanded.sort(key=lambda x: (x[0], x[1]))

    # ----------------------------------------------------------------
    # 3) Find LIS by page among the sorted items
    #
    #    We want strictly ascending item_order (which we have by sort),
    #    but for page, we often want non-decreasing or strictly ascending.
    #    For safety, let's do strictly ascending by page as well.
    #    If you want to allow same-page items, use <= instead of < in the LIS check.
    # ----------------------------------------------------------------
    n = len(expanded)
    dp = [1] * n  # dp[i] = length of the LIS ending at i
    parent = [-1] * n  # to reconstruct the path

    max_len = 0
    max_end_idx = 0

    for i in range(n):
        for j in range(i):
            # item_order is already sorted by expanded, so we only check page
            # We want strictly ascending page, so page_j < page_i
            if expanded[j][1] <= expanded[i][1]:
                if dp[j] + 1 > dp[i]:
                    dp[i] = dp[j] + 1
                    parent[i] = j
        if dp[i] > max_len:
            max_len = dp[i]
            max_end_idx = i
    
    # ----------------------------------------------------------------
    #    Reconstruct the LIS
    # ----------------------------------------------------------------
    lis_indices = set()
    cur = max_end_idx

    if cur == 0:
        return None

    while cur != -1:
        lis_indices.add(cur)
        cur = parent[cur]
    
    # ----------------------------------------------------------------
    #    Identify misplaced (those not in the LIS) and valid (in LIS)
    # ----------------------------------------------------------------
    valid_sequence = []
    misplaced_items = []
    for i, triple in enumerate(expanded):
        _, _, original_idx = triple
        if i in lis_indices:
            valid_sequence.append(items_list[original_idx])
        else:
            misplaced_items.append(items_list[original_idx])

    # Check for misplaced items that have a previous element with the same label in items_list
    for misplaced in misplaced_items[:]:
        misplaced_index = items_list.index(misplaced)
        if misplaced_index > 0:
            previous_item = items_list[misplaced_index - 1]
            if misplaced["label"] == previous_item["label"]:
                misplaced_items.remove(misplaced)

    # Sort misplaced items first by page, then by the value of their label converted to number using items_numbers
    misplaced_items.sort(key=lambda x: (x["page"], items_numbers.get(x["label"], 100)))
    if misplaced_items and misplaced_items[0]["label"] == "item01and02":
        misplaced_items.pop(0)
    if len(misplaced_items)==0:
        return None
    # Return them as two separate lists
    return misplaced_items


def best_fuzzy_match(text, synonyms, threshold=0.8):
    """
    Return the best fuzzy match score among synonyms for the given text.
    If no synonyms match above threshold, returns (None, 0.0).
    Otherwise returns (best_synonym, best_score).
    """
    best_synonym = None
    best_score = 0.0
    
    for syn in synonyms:
        score = fuzz.ratio(text, syn)
        score = score / 100.0  # Convert 0-100 to 0-1 scale
        if score > best_score:
            best_score = score
            best_synonym = syn
    
    if best_score >= threshold:
        return best_synonym
    else:
        return None


def check_start(row):
    """
    Simple heuristic: check if text is short enough or starts with 'item' or 'part'
    """
    # For now, just check if it starts with "item"
    start_text = row['clean_text']
    start_contidion = (start_text.startswith('item') or start_text.startswith('part'))
    return start_contidion


def first_search(df, ITEMS_STANDARD=ITEMS_STANDARD, print_results=False):
    # if print_results:
    #     print('ID:', df.doc_id.unique())
    #     print()

    items_list = []
    for label, references in ITEMS_STANDARD.items():
        for _, row in df[(~df.is_noise)&(df['clean_text'].str.find('item')!=-1)].iterrows():
            text = row['clean_text']
            fuzz_best_match = best_fuzzy_match(text, references)
            if fuzz_best_match:
                if check_start(row):
                    items_list.append({'label': label, 'page': row.page_number, 'id': row.name, 'text': row['md_text'], 'index': row.name})

    labels = [i['label'] for i in items_list]

    missing_items = [i for i in ITEMS.keys() if i not in labels]
    if any(i in labels for i in ['item01', 'item01c', 'item02']):
        if 'item01and02' in missing_items:
            missing_items.remove('item01and02')
    elif 'item01and02' in labels:
        missing_items = [i for i in missing_items if i not in ['item01', 'item01c', 'item02']]

    duplicate_items = [i for i in labels if labels.count(i)>1]
    duplicate_items = sorted(list(set(duplicate_items)))
    
    # Remove unescaped toc
    if duplicate_items:
        duplicate_pages = [i['page'] for i in items_list if i['label'] in duplicate_items]
        duplicate_pages = (list(set(duplicate_pages)))

        all_pages = sorted(list(set([i['page'] for i in items_list])))

        min_duplicate_page = min(duplicate_pages)

        if min_duplicate_page == min(all_pages) and min_duplicate_page < 7:
            page_text = ' '.join(df.loc[df['page_number'] == min_duplicate_page, 'clean_text'].astype(str).tolist())
            if page_text.lower().count('item') >= 8:
                print(f"Removing toc at page {min_duplicate_page}")
                df.loc[df['page_number'] == min_duplicate_page, ['is_noise', 'noise_type']] = [True, 'toc']
                duplicate_pages = [page for page in duplicate_pages if page != min_duplicate_page]
                items_list = [i for i in items_list if i['page'] != min_duplicate_page]
                labels = [i['label'] for i in items_list]
                duplicate_items = [i for i in labels if labels.count(i)>1]
                duplicate_items = sorted(list(set(duplicate_items)))



    misplaced_items = []
    
    misplaced =  find_misplaced_items(items_list)

    if misplaced:
        # Remove misplaced items that have a label present in duplicate_items from items_list
        for misplaced_item in misplaced:
            target_label = misplaced_item['label']
            if target_label in duplicate_items:
                items_list = [item for item in items_list if item['id'] != misplaced_item['id']]
                # list of all labels in items_list
                labels = [i['label'] for i in items_list]
                # if only one entry for target_label, remove it from duplicate_items
                if labels.count(target_label) == 1:
                    duplicate_items.remove(target_label)
                # remove target_label from misplaced
                misplaced.remove(misplaced_item)
        if misplaced:
            misplaced_items = [{'label': i['label'], 'id': i['id']} for i in misplaced]
    else:
        misplaced = []

    # Update duplicate_items list
    for label in duplicate_items[:]:
        if labels.count(label) == 1:
            duplicate_items.remove(label)

    # Check for specific items in the clean_text of the dataframe
    if 'item01c' in missing_items:
        search_terms = ['item 1c', 'item 1 c ', 'item 1c.', 'item 1.c ']
        found = any(df['clean_text'].str.contains('|'.join(search_terms), case=False, na=False))
        if not found:
            missing_items.remove('item01c')

    if 'item09c' in missing_items:
        search_terms = ['item 9c', 'item 9 c ', 'item 9c.', 'item 9.c ']
        found = any(df['clean_text'].str.contains('|'.join(search_terms), case=False, na=False))
        if not found:
            missing_items.remove('item09c')

    if 'item16' in missing_items:
        search_terms = ['item 16']
        found = any(df['clean_text'].str.contains('|'.join(search_terms), case=False, na=False))
        if not found:
            missing_items.remove('item16')


    if print_results:
        print('missing:', missing_items)
        print()   
        print('duplicates:', duplicate_items)
        print()
        print('misplaced:', misplaced)
        print()

    full_list = {'missing': missing_items, 'duplicates': duplicate_items, 'misplaced': misplaced_items, 'detected': items_list}

    return full_list




In [4]:
test_folder_path = 'outputs/train'

filenames_test = [f for f in os.listdir(test_folder_path) if f.startswith('t')]
filenames_test = sorted(filenames_test)

dfs = []

for filename in filenames_test:
    print(f"Document ID: {filename}")

    df = pd.read_parquet(f'{test_folder_path}/{filename}/df_toc.parquet')

    df = df[~df['clean_text'].isin(['part i', 'part ii', 'part iii', 'part iv'])]

    df['md_text'] = df['md_text'].apply(lambda x: re.sub(r'\*\*\s+\*\*', '', x))

    result = first_search(df, print_results=True)
    detected = result['detected']

    df['item'] = ''

    if detected:
        for item in detected:
            df.at[item['id'], 'item'] = item['label']

    # Json dumo results
    with open(f'{test_folder_path}/{filename}/item_fuzz.json', 'w') as f:
        json.dump(result, f, indent=4)

    df.to_parquet(f'{test_folder_path}/{filename}/df_item_fuzz.parquet')
    display(df)
    print('-'*50)

    dfs.append(df)



Document ID: tr00_0000001800_0001047469-15-001377
missing: []

duplicates: []

misplaced: []



,doc_id,page_number,pos_in_page,md_text,clean_text,item,type,emb,is_noise,noise_type,detected_page_number
id,,,,,,,,,,,
10,tr00_0000001800_0001047469-15-001377,0,0,10-K 1 a2222655z10-k.htm 10-K,10 k 1 a2222655z10 k.htm 10 k,,NaN,NaN,True,cover,NaN
20,tr00_0000001800_0001047469-15-001377,0,1,**UNITED STATES**,united states,,NaN,NaN,True,cover,NaN
30,tr00_0000001800_0001047469-15-001377,0,2,**SECURITIES AND EXCHANGE COMMISSION**,securities and exchange commission,,NaN,NaN,True,cover,NaN
40,tr00_0000001800_0001047469-15-001377,0,3,"**WASHINGTON, D. C. 20549**","washington, d. c. 20549",,NaN,NaN,True,cover,NaN
50,tr00_0000001800_0001047469-15-001377,0,4,**FORM 10-K**,form 10 k,,NaN,NaN,True,cover,NaN
...,...,...,...,...,...,...,...,...,...,...,...
15840,tr00_0000001800_0001047469-15-001377,105,29,The 2015 Abbott Laboratories Proxy Statement w...,the 2015 abbott laboratories proxy statement w...,,NaN,NaN,False,None,105.0
15850,tr00_0000001800_0001047469-15-001377,105,30,- Incorporated herein by reference. Commission...,incorporated herein by reference. commission f...,,NaN,NaN,False,None,105.0
15860,tr00_0000001800_0001047469-15-001377,105,31,** Denotes management contract or compensatory...,denotes management contract or compensatory pl...,,NaN,NaN,False,None,105.0


--------------------------------------------------
Document ID: tr01_0000002488_0000002488-16-000111
missing: []

duplicates: []

misplaced: []



,doc_id,page_number,pos_in_page,md_text,clean_text,item,type,emb,is_noise,noise_type,detected_page_number
id,,,,,,,,,,,
10,tr01_0000002488_0000002488-16-000111,0,0,10-K 1 amd-12262015x10k.htm 10-K,10 k 1 amd 12262015x10k.htm 10 k,,NaN,NaN,True,cover,NaN
20,tr01_0000002488_0000002488-16-000111,0,1,**(Mark One)**,(mark one),,NaN,NaN,True,cover,NaN
30,tr01_0000002488_0000002488-16-000111,0,2,**UNITED STATES**,united states,,NaN,NaN,True,cover,NaN
40,tr01_0000002488_0000002488-16-000111,0,3,**SECURITIES AND EXCHANGE COMMISSION**,securities and exchange commission,,NaN,NaN,True,cover,NaN
50,tr01_0000002488_0000002488-16-000111,0,4,"**Washington, D.C. 20549**","washington, d.c. 20549",,NaN,NaN,True,cover,NaN
...,...,...,...,...,...,...,...,...,...,...,...
21980,tr01_0000002488_0000002488-16-000111,106,12,Years ended:,years ended:,,NaN,NaN,False,None,105.0
21990,tr01_0000002488_0000002488-16-000111,106,13,"December 28, 2013 $ 2 $ (2) $ — $ —","december 28, 2013 $ 2 $ (2) $ — $ —",,NaN,NaN,False,None,105.0
22000,tr01_0000002488_0000002488-16-000111,106,14,"December 27, 2014 $ — $ — $ — $ —","december 27, 2014 $ — $ — $ — $ —",,NaN,NaN,False,None,105.0


--------------------------------------------------
Document ID: tr02_0000004127_0000004127-20-000058
missing: []

duplicates: []

misplaced: []



...

--------------------------------------------------
Document ID: tr98_0001590955_0001564590-20-004636
missing: ['item09b']

duplicates: []

misplaced: []



,doc_id,page_number,pos_in_page,md_text,clean_text,item,type,emb,is_noise,noise_type,detected_page_number
id,,,,,,,,,,,
10,tr98_0001590955_0001564590-20-004636,0,0,**UNITED STATES**,united states,,NaN,NaN,True,cover,NaN
20,tr98_0001590955_0001564590-20-004636,0,1,**SECURITIES AND EXCHANGE COMMISSION**,securities and exchange commission,,NaN,NaN,True,cover,NaN
30,tr98_0001590955_0001564590-20-004636,0,2,"**Washington, D.C. 20549**","washington, d.c. 20549",,NaN,NaN,True,cover,NaN
40,tr98_0001590955_0001564590-20-004636,0,3,**FORM 10-K**,form 10 k,,NaN,NaN,True,cover,NaN
50,tr98_0001590955_0001564590-20-004636,0,4,**(Mark One)**,(mark one),,NaN,NaN,True,cover,NaN
...,...,...,...,...,...,...,...,...,...,...,...
11750,tr98_0001590955_0001564590-20-004636,74,11,/s/ Janet Haugen\nJanet Haugen\nDirector,/s/ janet haugen janet haugen director,,NaN,NaN,False,None,73.0
11760,tr98_0001590955_0001564590-20-004636,74,12,/s/ Robert J. Levenson\nRobert J. Levenson\nDi...,/s/ robert j. levenson robert j. levenson dire...,,NaN,NaN,False,None,73.0
11770,tr98_0001590955_0001564590-20-004636,74,13,/s/ Frederick C. Peters II\nFrederick C. Peter...,/s/ frederick c. peters ii frederick c. peters...,,NaN,NaN,False,None,73.0


--------------------------------------------------
Document ID: tr99_0001601046_0001601046-22-000161
missing: []

duplicates: []

misplaced: []



,doc_id,page_number,pos_in_page,md_text,clean_text,item,type,emb,is_noise,noise_type,detected_page_number
id,,,,,,,,,,,
10,tr99_0001601046_0001601046-22-000161,0,0,**UNITED STATES**,united states,,NaN,NaN,True,cover,NaN
20,tr99_0001601046_0001601046-22-000161,0,1,**SECURITIES AND EXCHANGE COMMISSION**,securities and exchange commission,,NaN,NaN,True,cover,NaN
30,tr99_0001601046_0001601046-22-000161,0,2,"**Washington, D.C. 20549**","washington, d.c. 20549",,NaN,NaN,True,cover,NaN
40,tr99_0001601046_0001601046-22-000161,0,3,**Form 10-K**,form 10 k,,NaN,NaN,True,cover,NaN
50,tr99_0001601046_0001601046-22-000161,0,4,**(Mark One)**,(mark one),,NaN,NaN,True,cover,NaN
...,...,...,...,...,...,...,...,...,...,...,...
18230,tr99_0001601046_0001601046-22-000161,109,23,"/s/ ROBERT A. RANGO Director December 15, 2022","/s/ robert a. rango director december 15, 2022",,NaN,NaN,False,None,109.0
18240,tr99_0001601046_0001601046-22-000161,109,24,**Robert A. Rango**,robert a. rango,,NaN,NaN,False,None,109.0
18250,tr99_0001601046_0001601046-22-000161,109,25,"/s/ KEVIN A. STEPHENS Director December 15, 2022","/s/ kevin a. stephens director december 15, 2022",,NaN,NaN,False,None,109.0


--------------------------------------------------


In [5]:
test_folder_path = 'outputs/test'

filenames_test = [f for f in os.listdir(test_folder_path) if f.startswith('t')]
filenames_test = sorted(filenames_test)

dfs = []

for filename in filenames_test:
    print(f"Document ID: {filename}")

    df = pd.read_parquet(f'{test_folder_path}/{filename}/df_toc.parquet')

    df = df[~df['clean_text'].isin(['part i', 'part ii', 'part iii', 'part iv'])]

    df['md_text'] = df['md_text'].apply(lambda x: re.sub(r'\*\*\s+\*\*', '', x))


    result = first_search(df, print_results=True)
    detected = result['detected']

    df['item'] = ''

    if detected:
        for item in detected:
            df.at[item['id'], 'item'] = item['label']

    # Json dumo results
    with open(f'{test_folder_path}/{filename}/item_fuzz.json', 'w') as f:
        json.dump(result, f, indent=4)
    

    df.to_parquet(f'{test_folder_path}/{filename}/df_item_fuzz.parquet')
    
    print('-'*50)

    dfs.append(df)



Document ID: ts00_0000004127_0000004127-13-000056
missing: []

duplicates: []

misplaced: []

--------------------------------------------------
Document ID: ts01_0000004127_0000004127-18-000046
missing: []

duplicates: []

misplaced: []

--------------------------------------------------
Document ID: ts02_0000004447_0001628280-22-004524
missing: []

duplicates: []

misplaced: []

--------------------------------------------------
Document ID: ts03_0000010456_0001193125-14-061654
missing: []

duplicates: []

misplaced: []

--------------------------------------------------
Document ID: ts04_0000010456_0001628280-22-003432
missing: []

duplicates: []

misplaced: []

--------------------------------------------------
Document ID: ts05_0000018230_0000018230-19-000034
missing: ['item01c']

duplicates: []

misplaced: []

--------------------------------------------------
Document ID: ts06_0000021076_0001206774-14-002682
missing: []

duplicates: []

misplaced: []

---------------------------

In [ ]:
data = [
    {'doc': 'ts05', 'fuzzy_missing': 1, 'fuzzy_duplicates': 0, 'llm_missing': 1,  'llm_duplicates': 0},
    {'doc': 'ts11', 'fuzzy_missing': 0, 'fuzzy_duplicates': 2, 'llm_missing': 0,  'llm_duplicates': 0},
    {'doc': 'ts16', 'fuzzy_missing': 2, 'fuzzy_duplicates': 0, 'llm_missing': 0,  'llm_duplicates': 0},
    {'doc': 'ts18', 'fuzzy_missing': 11,'fuzzy_duplicates': 0, 'llm_missing': 0,  'llm_duplicates': 0},
    {'doc': 'ts26', 'fuzzy_missing': 20,'fuzzy_duplicates': 0, 'llm_missing': 16, 'llm_duplicates': 0},
    {'doc': 'ts33', 'fuzzy_missing': 6, 'fuzzy_duplicates': 0, 'llm_missing': 0,  'llm_duplicates': 0},
    {'doc': 'ts34', 'fuzzy_missing': 2, 'fuzzy_duplicates': 0, 'llm_missing': 0,  'llm_duplicates': 0},
    {'doc': 'ts39', 'fuzzy_missing': 0, 'fuzzy_duplicates': 1, 'llm_missing': 0,  'llm_duplicates': 0},
    {'doc': 'ts40', 'fuzzy_missing': 1, 'fuzzy_duplicates': 0, 'llm_missing': 0,  'llm_duplicates': 0},
    {'doc': 'ts42', 'fuzzy_missing': 2, 'fuzzy_duplicates': 0, 'llm_missing': 0,  'llm_duplicates': 0},
    {'doc': 'ts45', 'fuzzy_missing': 2, 'fuzzy_duplicates': 0, 'llm_missing': 0,  'llm_duplicates': 0},
]

# Sort by total Fuzzy errors (descending)
for d in data:
    d['fuzzy_total'] = d['fuzzy_missing'] + d['fuzzy_duplicates']
data_sorted = sorted(data, key=lambda x: x['fuzzy_total'], reverse=True)

doc_list             = [d['doc'] for d in data_sorted]
fuzzy_missing_list   = [d['fuzzy_missing']    for d in data_sorted]
fuzzy_duplicates_list= [d['fuzzy_duplicates'] for d in data_sorted]
llm_missing_list     = [d['llm_missing']      for d in data_sorted]
llm_duplicates_list  = [d['llm_duplicates']   for d in data_sorted]

# 3) Create grouped + stacked bars using offsetgroup and base
trace_fuzzy_missing = go.Bar(
    x=doc_list,
    y=fuzzy_missing_list,
    name='Fuzzy - Missing',
    marker_color='orange',
    offsetgroup='fuzzy'        # group Fuzzy bars together
)

trace_fuzzy_duplicates = go.Bar(
    x=doc_list,
    y=fuzzy_duplicates_list,
    name='Fuzzy - Duplicates',
    marker_color='red',
    offsetgroup='fuzzy',
    base=fuzzy_missing_list     # stack on top of "Fuzzy - Missing"
)

trace_llm_missing = go.Bar(
    x=doc_list,
    y=llm_missing_list,
    name='Fuzzy+LLM - Missing',
    marker_color='darkred',
    offsetgroup='llm'          # group LLM bars together
)

trace_llm_duplicates = go.Bar(
    x=doc_list,
    y=llm_duplicates_list,
    name='Fuzzy+LLM - Duplicates',
    marker_color='red',
    offsetgroup='llm',
    base=llm_missing_list       # stack on top of "Fuzzy+LLM - Missing"
)

fig = go.Figure(data=[
    trace_fuzzy_duplicates,
    trace_fuzzy_missing,
    trace_llm_missing,

])

# 4) Layout: 'group' mode => side-by-side groups;
#    using 'base=' => internal stacking within each group
fig.update_layout(
    barmode='group',
    title='Comparison of Fuzzy vs. Fuzzy+LLM Errors ',
    xaxis_title='Document\n',
    yaxis_title='Number of Errors',
    legend=dict(
        orientation='h',       # horizontal legend
        yanchor='bottom',
        y=-0.25,               # move it below the plot
        xanchor='center',
        x=0.5
    )
)

fig.show()
